In [18]:
import pandas as pd
import torch
from sklearn.utils import shuffle

In [19]:
import pandas as pd
rnames = ['userId', 'movieId', 'rating', "TimeStamp"]
ratings = pd.read_table("ml-1m.inter", header=0, names=rnames, engine='python')
ratings

,userId,movieId,rating,TimeStamp
0,1,1193,5,978300760
1,1,661,3,978302109
2,1,914,3,978301968
3,1,3408,4,978300275
4,1,2355,5,978824291
...,...,...,...,...
1000204,6040,1091,1,956716541
1000205,6040,1094,5,956704887
1000206,6040,562,5,956704746
1000207,6040,1096,4,956715648


In [20]:
ratio = 0.8
ratings = shuffle(ratings)
# ratings
train = ratings.copy()
test = ratings.copy()
train.iloc[int(ratio*len(ratings)):,2] = None
# train.iloc[int(ratio*len(ratings)):,'rating'] = 0
test.iloc[:int(ratio*len(ratings)),2] = None

In [23]:
ratings.describe()

,userId,movieId,rating,TimeStamp
count,1.000209e+06,1.000209e+06,1.000209e+06,1.000209e+06
mean,3.024512e+03,1.865540e+03,3.581564e+00,9.722437e+08
std,1.728413e+03,1.096041e+03,1.117102e+00,1.215256e+07
min,1.000000e+00,1.000000e+00,1.000000e+00,9.567039e+08
25%,1.506000e+03,1.030000e+03,3.000000e+00,9.653026e+08
50%,3.070000e+03,1.835000e+03,4.000000e+00,9.730180e+08
75%,4.476000e+03,2.770000e+03,4.000000e+00,9.752209e+08
max,6.040000e+03,3.952000e+03,5.000000e+00,1.046455e+09


In [24]:
rating_matrix = train.pivot(index='userId', columns='movieId', values='rating')
n_users, n_movies = rating_matrix.shape
# Scaling ratings to between 0 and 1, this helps our model by constraining predictions
min_rating, max_rating = train['rating'].min(), train['rating'].max()
rating_matrix = (rating_matrix - min_rating) / (max_rating - min_rating)
print(rating_matrix)
print(n_users*n_movies-rating_matrix.isnull().values.sum())

movieId  1     2     3     4     5     6     7     8     9     10    ...   
userId                                                               ...   
1         1.0   NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN  ...  \
2         NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN  ...   
3         NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN  ...   
4         NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN  ...   
5         NaN   NaN   NaN   NaN   NaN  0.25   NaN   NaN   NaN   NaN  ...   
...       ...   ...   ...   ...   ...   ...   ...   ...   ...   ...  ...   
6036      NaN   NaN   NaN  0.25   NaN  0.50   NaN   NaN   NaN   NaN  ...   
6037      NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN  ...   
6038      NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN  ...   
6039      NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN  ...   
6040      0.5   NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN  ...   

movieId  39

In [25]:
# Replacing missing ratings with -1 so we can filter them out later

rating_matrix[rating_matrix.isnull()] = -1
rating_matrix = torch.FloatTensor(rating_matrix.values)
print(rating_matrix)

tensor([[ 1.0000, -1.0000, -1.0000,  ..., -1.0000, -1.0000, -1.0000],
        [-1.0000, -1.0000, -1.0000,  ..., -1.0000, -1.0000, -1.0000],
        [-1.0000, -1.0000, -1.0000,  ..., -1.0000, -1.0000, -1.0000],
        ...,
        [-1.0000, -1.0000, -1.0000,  ..., -1.0000, -1.0000, -1.0000],
        [-1.0000, -1.0000, -1.0000,  ..., -1.0000, -1.0000, -1.0000],
        [ 0.5000, -1.0000, -1.0000,  ..., -1.0000, -1.0000, -1.0000]])


In [27]:
class Loss(torch.nn.Module):
    def __init__(self, lam_u=0.3, lam_v=0.3):
        super().__init__()
        self.lam_u = lam_u
        self.lam_v = lam_v
    
    def forward(self, matrix, u_features, v_features):
        non_zero_mask = (matrix != -1).type(torch.FloatTensor)
        predicted = torch.sigmoid(torch.mm(u_features, v_features.t()))
        
        diff = (matrix - predicted)**2
        prediction_error = torch.sum(diff*non_zero_mask)

        u_regularization = self.lam_u * torch.sum(u_features.norm(dim=1))
        v_regularization = self.lam_v * torch.sum(v_features.norm(dim=1))

        return prediction_error + u_regularization + v_regularization, prediction_error

In [34]:
# RFRecF: average movie_features 

alpha = 0.1             # learning rate 0.1, 0.01
threshold = 0.8         # prob of L2GD 0.8, 0.4
# lam = 10
# lr = alpha/(1-threshold)  
# eta = min(1,alpha*lam/threshold)
lr = 0.1
eta = 1       
num_client = 200        
m = n_users//num_client
num_epoch = 200         # 100
latent_vectors = 20    
# user_features = torch.randn(n_users, latent_vectors, requires_grad=True)

user_features = []
movie_features = []
for i in range(num_client):
    user_features.append(torch.randn(m, latent_vectors, requires_grad=True))
    movie_features.append(torch.randn(n_movies, latent_vectors, requires_grad=True))
with torch.no_grad():
    for i in range(num_client):
        # print(user_features[i])
        user_features[i].data.mul_(0.01)        # 0.1
        movie_features[i].data.mul_(0.01)       # 0.1
        # print(user_features[i])
    # print(movie_features[i])
RFRecF_error = Loss(lam_u=0.1, lam_v=0)       # lam_u=0.1, lam_v=0

optimizer_client_set = []
optimizer_user_set = []
optimizer_movie_set = []

for i in range(num_client):
    optimizer_client = torch.optim.Adam([user_features[i], movie_features[i]], lr=lr) 
    optimizer_client_set.append(optimizer_client)

aver_movie_features = torch.randn(n_movies, latent_vectors).data.mul_(0.01)
# aver_movie_features = torch.zeros(n_movies, latent_vectors).data.mul_(0.01)
error_list = []
previous = torch.zeros(n_movies, latent_vectors)
for step, epoch in enumerate(range(num_epoch)):
    aver_loss = 0
    aver_prediction_error = 0
    
    # tmp = torch.zeros(n_movies, latent_vectors)
    p = torch.rand(1)
    # print(p)
    if p>threshold:
        # local update
        for i in range(num_client):
            optimizer_client_set[i].zero_grad()
  
            loss, prediction_error = RFRecF_error(rating_matrix[i*m:(i+1)*m], user_features[i], movie_features[i])
            aver_loss += loss/num_client
            aver_prediction_error += prediction_error/num_client

            loss.backward()
            # for params in optimizer_client_set[i].param_groups:                      
            #     params['lr'] = 0.1 
            optimizer_client_set[i].step()
    else:
        # server update
        tmp = torch.zeros(n_movies, latent_vectors)
        with torch.no_grad():
            for i in range(num_client):
   
                # # laplace mechanism
                # s = 0.08
                # delta = 0.2
                # noise = torch.distributions.laplace.Laplace(torch.tensor([0.0]), torch.tensor([s])).sample()
                # tmp+= torch.clip(movie_features[i], min=-delta, max=delta) + noise

                # without perturbation
                tmp+= movie_features[i]

            aver_movie_features = tmp/num_client
            for i in range(num_client):
                movie_features[i] += eta*(aver_movie_features - movie_features[i]) 

    if step % 1 == 0:
        # print(f"Step {step}, {aver_loss:.3f}")
        print(f"Step {step}, {aver_prediction_error:.3f}")

Step 0, 0.000
Step 1, 0.000
Step 2, 0.000
Step 3, 0.000
Step 4, 0.000
Step 5, 394.320
Step 6, 0.000
Step 7, 0.000
Step 8, 0.000
Step 9, 0.000
Step 10, 0.000
Step 11, 0.000
Step 12, 0.000
Step 13, 0.000
Step 14, 0.000
Step 15, 0.000
Step 16, 0.000
Step 17, 0.000
Step 18, 394.547
Step 19, 0.000
Step 20, 0.000
Step 21, 0.000
Step 22, 393.857
Step 23, 0.000
Step 24, 0.000
Step 25, 385.541
Step 26, 0.000
Step 27, 358.374
Step 28, 0.000
Step 29, 308.832
Step 30, 0.000
Step 31, 0.000
Step 32, 0.000
Step 33, 260.472
Step 34, 0.000
Step 35, 0.000
Step 36, 0.000
Step 37, 0.000
Step 38, 0.000
Step 39, 0.000
Step 40, 0.000
Step 41, 0.000
Step 42, 0.000
Step 43, 0.000
Step 44, 0.000
Step 45, 243.459
Step 46, 0.000
Step 47, 0.000
Step 48, 0.000
Step 49, 0.000
Step 50, 245.197
Step 51, 0.000
Step 52, 245.393
Step 53, 0.000
Step 54, 240.509
Step 55, 0.000
Step 56, 0.000
Step 57, 0.000
Step 58, 233.677
Step 59, 0.000
Step 60, 0.000
Step 61, 0.000
Step 62, 0.000
Step 63, 0.000
Step 64, 0.000
Step 65, 0.

In [36]:
test_rating_matrix = test.pivot(index='userId', columns='movieId', values='rating')

test_rating_matrix[test_rating_matrix.isnull()] = -1
test_rating_matrix = torch.FloatTensor(test_rating_matrix.values)
# test_rating_matrix

In [38]:
non_zero_mask = (test_rating_matrix != -1).type(torch.FloatTensor)
num = torch.sum(non_zero_mask)
def Error(matrix, u_features, v_features):
    predicted_ratings = torch.sigmoid(torch.mm(u_features,v_features.t()))
    # predicted_ratings = non_zero_mask * predicted_ratings
    pred = (predicted_ratings*(max_rating - min_rating) + min_rating)*non_zero_mask[i*m:(i+1)*m]
    actual = matrix*non_zero_mask[i*m:(i+1)*m]
    AE_diff = torch.abs(pred - actual)
    SE_diff = (pred - actual)**2
    
    prediction_abs_error = torch.sum(AE_diff)
    prediction_squared_error = torch.sum(SE_diff)
    n_non_zero = torch.sum(non_zero_mask[i*m:(i+1)*m])
    return prediction_abs_error, prediction_squared_error, n_non_zero

AE_error = 0
SE_error = 0
num_non_zero = 0
movie_features = aver_movie_features
for i in range(num_client):
    AE_error += Error(test_rating_matrix[i*m:(i+1)*m], user_features[i], movie_features)[0]
    SE_error += Error(test_rating_matrix[i*m:(i+1)*m], user_features[i], movie_features)[1]
    num_non_zero += Error(test_rating_matrix[i*m:(i+1)*m], user_features[i], movie_features)[2]
test_MAE = AE_error/num_non_zero
test_RMSE = torch.sqrt(SE_error/num_non_zero)
print('test_MAE =', test_MAE.data.numpy())
print('test_RMSE =', test_RMSE.data.numpy())

test_MAE = 0.6925099
test_RMSE = 0.8848876
